<a href="https://colab.research.google.com/github/ChaithanyaSaiB/UMBC-DATA606-Capstone/blob/main/notebooks/Dataset_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('words')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
cnnhealth = pd.read_csv("/content/cnnhealth.txt", sep=r"(?<!\s)[|](?!\s)", names=['ID', 'DateTime', 'Article Title and Link'], engine="python")
cnnhealth.head()

,ID,DateTime,Article Title and Link
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...


In [ ]:
cnnhealth['URL'] = [re.findall(r'https?://\S+', text) for text in cnnhealth['Article Title and Link']]
cnnhealth.head(2)

,ID,DateTime,Article Title and Link,URL
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,[http://cnn.it/1L1t1Fv]
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...,"[http://cnn.it/1xdpsjT, http://pbs.twimg.com/m..."


In [ ]:
cnnhealth['URL'].apply(len).value_counts()

1    3299
0     457
2     305
Name: URL, dtype: int64

In [ ]:
drop_indices = cnnhealth['Article Title and Link'][cnnhealth['URL'].apply(len) != 1].index
cnnhealth.drop(drop_indices, inplace=True)
cnnhealth['URL'] = [url[0] for url in cnnhealth['URL']]
cnnhealth.head()

,ID,DateTime,Article Title and Link,URL
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk
9,576464606551490560,Fri Mar 13 19:27:27 +0000 2015,CDC: Misuse of garments may have led to releas...,http://cnn.it/18HDKDg
17,576353658184450048,Fri Mar 13 12:06:34 +0000 2015,RT @cnnbrk: U.S. Ebola patient headed to Natio...,http://cnn.it/1BxKgIa.
20,576342990446288896,Fri Mar 13 11:24:11 +0000 2015,RT @CNN: .@RobertDowneyJr presented a child wi...,http://cnn.it/1CbnIPC


In [ ]:
cnnhealth_subset = cnnhealth[:1000].copy()

In [ ]:
import pandas as pd

# Function to fetch the content from a URL
def fetch_content(url):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print("Error fetching content:", e)
        return None

# Function to extract body content from HTML using BeautifulSoup
def extract_body(html_content):
    if html_content:
        soup = BeautifulSoup(html_content, 'html.parser')
        article_body = soup.findAll('p', class_='paragraph')
        if article_body:
            return article_body
        else:
            return None
    else:
        return None

# Apply web scraping to extract content and store it in a new column
cnnhealth_subset['Content'] = cnnhealth_subset['URL'].apply(lambda url: extract_body(fetch_content(url)))

# Print the DataFrame with the extracted content
cnnhealth_subset.head()


Error fetching content: Failed to parse: https://t…
Error fetching content: Failed to parse: http://t.c…
Error fetching content: Failed to parse: http://…
Error fetching content: Failed to parse: http://t.co…
Error fetching content: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Error fetching content: HTTPSConnectionPool(host='flipboard.com', port=443): Read timed out. (read timeout=10)
Error fetching content: HTTPSConnectionPool(host='flipboard.com', port=443): Read timed out. (read timeout=10)
Error fetching content: HTTPSConnectionPool(host='flipboard.com', port=443): Read timed out. (read timeout=10)
Error fetching content: Failed to parse: http://…
Error fetching content: HTTPSConnectionPool(host='flipboard.com', port=443): Read timed out. (read timeout=10)
Error fetching content: HTTPSConnectionPool(host='flipboard.com', port=443): Read timed out. (read timeout=10)
Error fetching content: Failed to parse: http://t.co…
Error fetching content: HTTPC

,ID,DateTime,Article Title and Link,URL,Content
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv,"[[\n When I was a child, a pale spe..."
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk,[[\n “Hey dude – can you turn your ...
9,576464606551490560,Fri Mar 13 19:27:27 +0000 2015,CDC: Misuse of garments may have led to releas...,http://cnn.it/18HDKDg,[[\n The misuse of outer protective...
17,576353658184450048,Fri Mar 13 12:06:34 +0000 2015,RT @cnnbrk: U.S. Ebola patient headed to Natio...,http://cnn.it/1BxKgIa.,[[\n The National Institutes of Hea...
20,576342990446288896,Fri Mar 13 11:24:11 +0000 2015,RT @CNN: .@RobertDowneyJr presented a child wi...,http://cnn.it/1CbnIPC,[[\n Robert Downey Jr. may be Iron ...


In [ ]:
cnnhealth_subset.isnull().sum()

ID                          0
DateTime                    0
Article Title and Link      0
URL                         0
Content                   351
dtype: int64

In [ ]:
cnnhealth_subset.dropna(inplace=True)

In [ ]:
cnnhealth_subset['Content'] = [" ".join([p_tag.text.strip() for p_tag in content]) for content in cnnhealth_subset['Content']]
cnnhealth_subset

,ID,DateTime,Article Title and Link,URL,Content
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv,"When I was a child, a pale specter used to cal..."
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk,“Hey dude – can you turn your music down?” If ...
9,576464606551490560,Fri Mar 13 19:27:27 +0000 2015,CDC: Misuse of garments may have led to releas...,http://cnn.it/18HDKDg,The misuse of outer protective garments may ha...
17,576353658184450048,Fri Mar 13 12:06:34 +0000 2015,RT @cnnbrk: U.S. Ebola patient headed to Natio...,http://cnn.it/1BxKgIa.,The National Institutes of Health said Thursda...
20,576342990446288896,Fri Mar 13 11:24:11 +0000 2015,RT @CNN: .@RobertDowneyJr presented a child wi...,http://cnn.it/1CbnIPC,Robert Downey Jr. may be Iron Man in the popul...
...,...,...,...,...,...
1515,420665054020001792,Tue Jan 07 21:15:39 +0000 2014,#DASH wins again for @USNewsHealth #BestDiets ...,http://cnn.it/1aDUt50,Followers of the Paleo Diet may go prehistoric...
1517,420590747625459712,Tue Jan 07 16:20:23 +0000 2014,"We don't say someone is ""cardiovascular dead"" ...",http://cnn.it/1a3Yf7T,A person who is brain dead may appear alive – ...
1520,420570933909549056,Tue Jan 07 15:01:39 +0000 2014,"Uh-oh, @Crossfit -- @USNewsHealth ranked #Pale...",http://cnn.it/1aDUt50,Followers of the Paleo Diet may go prehistoric...
1522,420344738676744194,Tue Jan 07 00:02:49 +0000 2014,. @FSU_Football coach is ready to tackle @Foot...,http://cnn.it/1esAUzn,Ethan Fisher is small for his age. The 8-year-...


In [ ]:
cnnhealth_subset.to_csv('data.csv', index=False)

In [3]:
cnnhealth = pd.read_csv('/content/data.csv')
cnnhealth.head()

,ID,DateTime,Article Title and Link,URL,Content
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv,"When I was a child, a pale specter used to cal..."
1,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk,“Hey dude – can you turn your music down?” If ...
2,576464606551490560,Fri Mar 13 19:27:27 +0000 2015,CDC: Misuse of garments may have led to releas...,http://cnn.it/18HDKDg,The misuse of outer protective garments may ha...
3,576353658184450048,Fri Mar 13 12:06:34 +0000 2015,RT @cnnbrk: U.S. Ebola patient headed to Natio...,http://cnn.it/1BxKgIa.,The National Institutes of Health said Thursda...
4,576342990446288896,Fri Mar 13 11:24:11 +0000 2015,RT @CNN: .@RobertDowneyJr presented a child wi...,http://cnn.it/1CbnIPC,Robert Downey Jr. may be Iron Man in the popul...


In [19]:
data = cnnhealth.Content.copy()
data[0]

'When I was a child, a pale specter used to call our house most evenings, eager to chat with my doctor father about her myriad medical concerns. We called her the “White Bread Lady,” a moniker she earned for one particularly inane call in which she panicked to my father after consuming white bread. She wasn’t breaking out in hives or having any adverse effects to the bread. No, she was just concerned that some future illness could befall her given that one particular dietary decision. Although we all laughed at the time, it was with a bit of shifty-eyed shame. Because most of us (including if not particularly the illustrious Ehrlich family) have lurking within us our very own “White Bread Lady,” ready to convince us that each cough, sniffle and less-than-nutritious meal might be a detriment to our health. And, naturally, that White Bread Lady looms even larger when we can type our worries into a search bar and unlock a bevy of potentially distressing information. Yup, so quoth Google, 

In [20]:
data = data.apply(word_tokenize)

In [21]:
def split_string_with_special_characters(word_list):
  # Define pattern for special characters
  pattern = r'[-\s]'

  for word in word_list:
    if re.search(r'[-\s]', word):
      # Split the string using the pattern
      substrings = re.split(pattern, word)

      # Remove empty substrings
      substrings = [substr for substr in substrings if substr]

      word_list.remove(word)
      word_list.extend(substrings)
  return word_list

# Apply the function to your data
data = data.apply(lambda word_list: split_string_with_special_characters(word_list))
print(data[0])

['When', 'I', 'was', 'a', 'child', ',', 'a', 'pale', 'specter', 'used', 'to', 'call', 'our', 'house', 'most', 'evenings', ',', 'eager', 'to', 'chat', 'with', 'my', 'doctor', 'father', 'about', 'her', 'myriad', 'medical', 'concerns', '.', 'We', 'called', 'her', 'the', '“', 'White', 'Bread', 'Lady', ',', '”', 'a', 'moniker', 'she', 'earned', 'for', 'one', 'particularly', 'inane', 'call', 'in', 'which', 'she', 'panicked', 'to', 'my', 'father', 'after', 'consuming', 'white', 'bread', '.', 'She', 'wasn', '’', 't', 'breaking', 'out', 'in', 'hives', 'or', 'having', 'any', 'adverse', 'effects', 'to', 'the', 'bread', '.', 'No', ',', 'she', 'was', 'just', 'concerned', 'that', 'some', 'future', 'illness', 'could', 'befall', 'her', 'given', 'that', 'one', 'particular', 'dietary', 'decision', '.', 'Although', 'we', 'all', 'laughed', 'at', 'the', 'time', ',', 'it', 'was', 'with', 'a', 'bit', 'of', 'shame', '.', 'Because', 'most', 'of', 'us', '(', 'including', 'if', 'not', 'particularly', 'the', 'ill

In [22]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def lowercase_words_and_lemmatize(word_list):
    # Convert words to lowercase
    word_list_lower = [word.lower() for word in word_list]

    # Tag POS for each word
    pos_tags = nltk.pos_tag(word_list_lower)

    # Map POS tags to WordNet POS tags
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN  # Default to noun if POS tag not found

    # Lemmatize words using correct POS tags
    lemmatized_words = []
    for word, tag in pos_tags:
        pos = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(word, pos=pos)
        lemmatized_words.append(lemma)

    return lemmatized_words

# Apply the function to your data
data = data.apply(lambda word_list: lowercase_words_and_lemmatize(word_list))
print(data[0])


['when', 'i', 'be', 'a', 'child', ',', 'a', 'pale', 'specter', 'use', 'to', 'call', 'our', 'house', 'most', 'evening', ',', 'eager', 'to', 'chat', 'with', 'my', 'doctor', 'father', 'about', 'her', 'myriad', 'medical', 'concern', '.', 'we', 'call', 'her', 'the', '“', 'white', 'bread', 'lady', ',', '”', 'a', 'moniker', 'she', 'earn', 'for', 'one', 'particularly', 'inane', 'call', 'in', 'which', 'she', 'panic', 'to', 'my', 'father', 'after', 'consume', 'white', 'bread', '.', 'she', 'wasn', '’', 't', 'break', 'out', 'in', 'hive', 'or', 'have', 'any', 'adverse', 'effect', 'to', 'the', 'bread', '.', 'no', ',', 'she', 'be', 'just', 'concern', 'that', 'some', 'future', 'illness', 'could', 'befall', 'her', 'give', 'that', 'one', 'particular', 'dietary', 'decision', '.', 'although', 'we', 'all', 'laugh', 'at', 'the', 'time', ',', 'it', 'be', 'with', 'a', 'bit', 'of', 'shame', '.', 'because', 'most', 'of', 'u', '(', 'include', 'if', 'not', 'particularly', 'the', 'illustrious', 'ehrlich', 'family'

In [23]:
parts_of_speech_to_remove = ['DT', 'IN', 'PRP', 'PRP$', 'CC', 'VB', 'JJ']

def stopwords_removal(word_list):
  # Tag words with their parts of speech
  tagged_words = nltk.pos_tag(word_list)

  # Get a list of common English stopwords
  stop_words = set(stopwords.words('english'))

  f = open('custom stopwords.txt','r')
  custom_stopwords = [word.strip() for word in f.readlines()]
  f.close()

  return [word for word, pos in tagged_words if pos not in parts_of_speech_to_remove and word not in stop_words and word not in custom_stopwords]

data = data.apply(lambda word_list: stopwords_removal(word_list))
data[0]

['child',
 ',',
 'specter',
 'house',
 ',',
 'father',
 'myriad',
 'concern',
 '.',
 'call',
 '“',
 'bread',
 'lady',
 ',',
 'moniker',
 'earn',
 'particularly',
 'inane',
 'call',
 'panic',
 'father',
 'bread',
 '.',
 '’',
 'break',
 'hive',
 'effect',
 'bread',
 '.',
 ',',
 'concern',
 'future',
 'illness',
 'could',
 'give',
 'decision',
 '.',
 'laugh',
 ',',
 'bit',
 'shame',
 '.',
 '(',
 'include',
 'particularly',
 'family',
 ')',
 'lurk',
 '“',
 'bread',
 'lady',
 ',',
 '”',
 'ready',
 'cough',
 ',',
 'sniffle',
 'meal',
 'might',
 'detriment',
 'health',
 '.',
 ',',
 'naturally',
 ',',
 'bread',
 'lady',
 'loom',
 'worry',
 'search',
 'bar',
 'bevy',
 'potentially',
 'information',
 '.',
 'yup',
 ',',
 'google',
 ',',
 'cancer',
 '.',
 'accord',
 'report',
 'research',
 'center',
 ',',
 '80',
 '%',
 'internet',
 'user',
 'look',
 'health',
 'information',
 'online',
 '.',
 'practice',
 'respect',
 ',',
 'abundance',
 '(',
 'variably',
 'valid',
 ')',
 'information',
 'online',


In [24]:
english_words = set(nltk.corpus.words.words())

def remove_non_alphabetic_and_custom_stopwords(word_list):
    # Remove non-alphabetic characters and filter out empty strings
    return [word for word in [re.sub(r'[^a-zA-Z]', '', word) for word in word_list] if word in english_words]

# Apply the function to your data
data = data.apply(lambda word_list: remove_non_alphabetic_and_custom_stopwords(word_list))
data[0]

['child',
 'specter',
 'house',
 'father',
 'myriad',
 'concern',
 'call',
 'bread',
 'lady',
 'moniker',
 'earn',
 'particularly',
 'inane',
 'call',
 'panic',
 'father',
 'bread',
 'break',
 'hive',
 'effect',
 'bread',
 'concern',
 'future',
 'illness',
 'could',
 'give',
 'decision',
 'laugh',
 'bit',
 'shame',
 'include',
 'particularly',
 'family',
 'lurk',
 'bread',
 'lady',
 'ready',
 'cough',
 'sniffle',
 'meal',
 'might',
 'detriment',
 'health',
 'naturally',
 'bread',
 'lady',
 'loom',
 'worry',
 'search',
 'bar',
 'bevy',
 'potentially',
 'information',
 'cancer',
 'accord',
 'report',
 'research',
 'center',
 'user',
 'look',
 'health',
 'information',
 'practice',
 'respect',
 'abundance',
 'variably',
 'valid',
 'information',
 'care',
 'need',
 'read',
 'five',
 'mistake',
 'courtesy',
 'selection',
 'make',
 'search',
 'eye',
 'twitch',
 'overly',
 'college',
 'student',
 'sit',
 'girl',
 'lecture',
 'hall',
 'pencil',
 'hop',
 'whiff',
 'hair',
 'type',
 'eye',
 'twi

In [25]:
# Join function to convert list of words to string
def join_words(word_list):
    return ' '.join(word_list)

data = data.apply(join_words)
data.head()

0    child specter house father myriad concern call...
1    hey dude music anyone say take note probably h...
2    misuse outer garment lead exposure potentially...
3    institute health say plan hospital health care...
4    man marvel superhero film recently technology ...
Name: Content, dtype: object

In [26]:
data.to_csv('cnnhealth_transformed.csv', index=False)